In [8]:
import pandas as pd


# df['raised_amount_usd'] = df['raised_amount_usd'].apply(lambda x: ("".join(x.split(","))).strip())
# df.loc[df.funding_total_usd == '-', 'raised_amount_usd'] = None
# df['raised_amount_usd'] = df['raised_amount_usd'].map(lambda x: float(x) if x is not None else x)


In [9]:
def split_backslash(x):
    try:
        return str(x).split("/")[1].strip()
    except:
        return x

In [10]:
df = pd.read_csv("../raw_data_crunchbase/crunchbase-investments.csv")
df = df[df['funded_year']==2013]
df['investor_permalink'] = df['investor_permalink'].apply(lambda x: split_backslash(x))
df = df.loc[df['investor_permalink']=='organization'] # change to person!!!!!!!

# from 20179 to 17617 for raised_amount_usd
df = df.loc[df["raised_amount_usd"].notnull()]
df['raised_amount_usd'] = df['raised_amount_usd'].apply(lambda x: "".join(x.split(",")).strip())
df.loc[df.raised_amount_usd == '-', 'raised_amount_usd'] = None

df['raised_amount_usd'] = df['raised_amount_usd'].map(lambda x: float(x) if x is not None else x)

# all present for funding_round_type
# df = df.loc[df["funding_round_type"].notnull()]
# from 17617 to 16553 for company_country_code
df = df.loc[df["investor_country_code"].notnull()]
df = df.loc[df["company_country_code"].notnull()]
# df = df.loc[df["investor_market"].notnull()]

In [11]:
df.shape

(11260, 24)

In [12]:
df.columns

Index([u'company_permalink', u'company_name', u'company_category_list',
       u'company_market', u'company_country_code', u'company_state_code',
       u'company_region', u'company_city', u'investor_permalink',
       u'investor_name', u'investor_category_list', u'investor_market',
       u'investor_country_code', u'investor_state_code', u'investor_region',
       u'investor_city', u'funding_round_permalink', u'funding_round_type',
       u'funding_round_code', u'funded_at', u'funded_month', u'funded_quarter',
       u'funded_year', u'raised_amount_usd'],
      dtype='object')

In [13]:
subset_df = df[["investor_country_code",
                "company_country_code",
                "raised_amount_usd"]]
# grouped = subset_df.groupby(['investor_name', 'investor_market'])
agg_df = subset_df.groupby(["investor_country_code","company_country_code"]).sum().reset_index()
print(agg_df.shape)
agg_df.head()

(436, 3)


,investor_country_code,company_country_code,raised_amount_usd
0,ARG,ARG,29117500
1,ARG,BRA,23550000
2,ARG,CHE,75000
3,ARG,CHL,11060000
4,ARG,COL,25000


In [26]:
# agg_wide = agg_df.pivot(index='investor_country_code', columns='company_country_code', values='raised_amount_usd')

agg_wide = pd.DataFrame(agg_df.set_index(['investor_country_code', 'company_country_code']).unstack('company_country_code').reset_index())
agg_wide = agg_wide.fillna(0)

In [27]:
agg_wide.head()

investor_country_code raised_amount_usd                \
company_country_code                                     ARE       ARG ARM   
0                                      ARG                 0  29117500   0   
1                                      AUS                 0         0   0   
2                                      AUT                 0         0   0   
3                                      AZE                 0         0   0   
4                                      BEL                 0         0   0   

                                                                 ...          \
company_country_code        AUS AUT     AZE       BEL BGD BGR    ...     SAU   
0                             0   0       0         0   0   0    ...       0   
1                     107565269   0       0         0   0   0    ...       0   
2                             0   0       0         0   0   0    ...       0   
3                             0   0  100000         0   0   0    ...       0   
4                      26000000   0       0  68790507   0   0    ...       0   

                                                                            
company_country_code       SGP SLV SVK SWE THA      TUR TWN UKR        USA  
0                            0   0   0   0   0        0   0   0     710000  
1                     19740000   0   0   0   0        0   0   0  416222833  
2                       800000   0   0   0   0  1801240   0   0          0  
3                            0   0   0   0   0        0   0   0          0  
4                            0   0   0   0   0        0   0   0   66800000  

[5 rows x 71 columns]

In [28]:
agg_df.to_csv("aggregated-long-country2country.csv", index=False)
agg_wide.to_csv("aggregated-wide-country2country.csv", index=False)